# ===========================================================
# 06 Working with Data

## Objectives
- Select individual values from a Pandas dataframe.
- Select entire rows or entire columns from a dataframe.
- Select a subset of both rows and columns from a dataframe in a single operation.
- Filter data based on values in a dataframe.
- Apply a split-apply-combine workflow to a dataframe.
- Convert a dataframe between wide and long data formats.

## Some Notes on DataFrames

- A DataFrame is a collection of Series
  - The DataFrame is the way Pandas represents a table, and Series is the data-structure
    Pandas use to represent a column.
- Pandas is built on top of the Numpy library
    - We get to use those function too
- Benefits of using Pandas:
  - Interface to access individual records
  - Proper handling of missing values
  - Relational-databases operations between DataFrames

## Selecting values
- Use `DataFrame.iloc[ROW, COLUMN]` to select values by their numerical position

In [1]:
import pandas as pd
data = pd.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.iloc[0, 0])

1601.056136


- Use `DataFrame.loc[..., ...]` to select values by their row/column labels.

In [2]:
data = pd.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.loc["Albania", "gdpPercap_1952"])

1601.056136


- Use `:` on its own to mean all columns or all rows.
  - This follows Python's slicing notation.

In [3]:
print(data.loc["Albania", :])

gdpPercap_1952    1601.056136
gdpPercap_1957    1942.284244
gdpPercap_1962    2312.888958
gdpPercap_1967    2760.196931
gdpPercap_1972    3313.422188
gdpPercap_1977    3533.003910
gdpPercap_1982    3630.880722
gdpPercap_1987    3738.932735
gdpPercap_1992    2497.437901
gdpPercap_1997    3193.054604
gdpPercap_2002    4604.211737
gdpPercap_2007    5937.029526
Name: Albania, dtype: float64


- We get the same result printing `data.loc["Albania"]` (without a second index).

In [4]:
print(data.loc[:, "gdpPercap_1952"])

country
Albania                    1601.056136
Austria                    6137.076492
Belgium                    8343.105127
Bosnia and Herzegovina      973.533195
Bulgaria                   2444.286648
Croatia                    3119.236520
Czech Republic             6876.140250
Denmark                    9692.385245
Finland                    6424.519071
France                     7029.809327
Germany                    7144.114393
Greece                     3530.690067
Hungary                    5263.673816
Iceland                    7267.688428
Ireland                    5210.280328
Italy                      4931.404155
Montenegro                 2647.585601
Netherlands                8941.571858
Norway                    10095.421720
Poland                     4029.329699
Portugal                   3068.319867
Romania                    3144.613186
Serbia                     3581.459448
Slovak Republic            5074.659104
Slovenia                   4215.041741
Spain            

- Would get the same result printing `data["gdpPercap_1952"]`
- Slicing works with labels as well as numerical positions

In [5]:
# 1. Use slicing to select GDP data from Italy to Poland
#    and from 1962 to 1972.j
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'])

             gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
country                                                    
Italy           8243.582340    10022.401310    12269.273780
Montenegro      4649.593785     5907.850937     7778.414017
Netherlands    12790.849560    15363.251360    18794.745670
Norway         13450.401510    16361.876470    18965.055510
Poland          5338.752143     6557.152776     8006.506993


In [6]:
print(data.loc[['Italy', 'Norway', 'Montenegro']])

            gdpPercap_1952  gdpPercap_1957  gdpPercap_1962  gdpPercap_1967  \
country                                                                      
Italy          4931.404155     6248.656232     8243.582340    10022.401310   
Norway        10095.421720    11653.973040    13450.401510    16361.876470   
Montenegro     2647.585601     3682.259903     4649.593785     5907.850937   

            gdpPercap_1972  gdpPercap_1977  gdpPercap_1982  gdpPercap_1987  \
country                                                                      
Italy         12269.273780    14255.984750     16537.48350     19207.23482   
Norway        18965.055510    23311.349390     26298.63531     31540.97480   
Montenegro     7778.414017     9595.929905     11222.58762     11732.51017   

            gdpPercap_1992  gdpPercap_1997  gdpPercap_2002  gdpPercap_2007  
country                                                                     
Italy         22013.644860    24675.024460    27968.098170    28

- Slicing using `loc` is inclusive at both ends
  - This differs from slicing using `iloc`  

## Putting it Together
- The result from slicing can be used in further operations.

In [7]:
# Find the maximum GDP for the above countries and above years
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'].max())

gdpPercap_1962    13450.40151
gdpPercap_1967    16361.87647
gdpPercap_1972    18965.05551
dtype: float64


In [8]:
# Find the minimum GDP for the above countries and above years
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'].min())

gdpPercap_1962    4649.593785
gdpPercap_1967    5907.850937
gdpPercap_1972    7778.414017
dtype: float64


## Filtering Data
- Use comparisons to select data based on values.
- Comparisons are applied element by element.
- They return a similarly-shaped dataframe of `True` and `False` values.

In [9]:
# Use a subset of data to keep output readable.
subset = data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972']
print('Subset of data:\n', subset)

# Which values were greater than 10000 ?
print('\nWhere are values large?\n', subset > 10000)

Subset of data:
              gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
country                                                    
Italy           8243.582340    10022.401310    12269.273780
Montenegro      4649.593785     5907.850937     7778.414017
Netherlands    12790.849560    15363.251360    18794.745670
Norway         13450.401510    16361.876470    18965.055510
Poland          5338.752143     6557.152776     8006.506993

Where are values large?
             gdpPercap_1962 gdpPercap_1967 gdpPercap_1972
country                                                 
Italy                False           True           True
Montenegro           False          False          False
Netherlands           True           True           True
Norway                True           True           True
Poland               False          False          False


- A dataframe full of Booleans is sometimes called a *mask* because of how it can be used.

In [10]:
mask = subset > 10000
print(subset[mask])

             gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
country                                                    
Italy                   NaN     10022.40131     12269.27378
Montenegro              NaN             NaN             NaN
Netherlands     12790.84956     15363.25136     18794.74567
Norway          13450.40151     16361.87647     18965.05551
Poland                  NaN             NaN             NaN


- NaNs (Not a Number) are ignored by operations like max, min, average, etc.

In [11]:
print(subset[subset > 10000].describe())

       gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
count        2.000000        3.000000        3.000000
mean     13120.625535    13915.843047    16676.358320
std        466.373656     3408.589070     3817.597015
min      12790.849560    10022.401310    12269.273780
25%      12955.737547    12692.826335    15532.009725
50%      13120.625535    15363.251360    18794.745670
75%      13285.513523    15862.563915    18879.900590
max      13450.401510    16361.876470    18965.055510


## Exercise: Split-Apply-Combine

### Group By

- A common data-wrangling technique is the split-apply-combine technique
![Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/_images/06_groupby1.svg)

Source: <https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/06_calculate_statistics.html#min-tut-06-stats>

## GDP of European Countries
We will practice this technique with the Gapminder data. Suppose we want to have a clearer view on how the European countries split themselves according to their GDP.

1.  We will split the countries in two groups during the years surveyed,
    those who presented a GDP *higher* than the European average and those with a *lower* GDP.
2.  We then estimate a *wealthy score* based on the historical (from 1962 to 2007) values,
    where we count how many times a country participated in the higher GDP group and give a percentage score.
3.  We will determine the totol GDP contribution for each year using wealth scores as categories.

**Part I - Split/Apply**

Type the following code and run it.

```python
wealth_score = data[data > data.mean()].count(axis=1) / len(data.columns)
wealth_score
```

- We filtered all the countris that were above the mean GDP.
- We then counted all the non-NaN values.
  - Our filter used NaN's for years a country was equal to or less than the mean.
- `axis` tells `count()` whether or not to count across rows or columns.
  - `0` means count non-NaN's in each column
  - `1` means count non-NaN's in each row (what we did)
  - I like to think about what is being collapsed (see picture below)
  - ... or just memorize 0 = down, 1 = across
- Finally, we divided the counts by the number of years (columns) to get a percentage score,

![axis](https://i.stack.imgur.com/DL0iQ.jpg)

Source: <https://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition#answers>

**Part II - Combine**

Use `groupby()` to sum the financial contribution of wealthy countries in different wealth-score categories across the years surveyed by typing the following code and running it.

```python
data.groupby(wealth_score).sum()
```

## Tidy Data
- [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf) principles are guidlines for organizing data that makes analysis on a computer more efficient and effective.
- Keep the following principles in mind when organizing your data:
  - Each variable has its own column
  - Each observation has its own row
  - Each value must have its own cell (atomic values)
  - Each type of observational unit forms a table

## Long and Wide Formats
- Pandas has several functions that help us rearrange our data when we need to change its structure.
  - This frequently occurs when we need to plot our data.
- When data is in *wide format*, each variable has its own column.
  - Our data is currently in *wide format*

In [12]:
data.head()

gdpPercap_1952  gdpPercap_1957  gdpPercap_1962  \
country                                                                  
Albania                    1601.056136     1942.284244     2312.888958   
Austria                    6137.076492     8842.598030    10750.721110   
Belgium                    8343.105127     9714.960623    10991.206760   
Bosnia and Herzegovina      973.533195     1353.989176     1709.683679   
Bulgaria                   2444.286648     3008.670727     4254.337839   

                        gdpPercap_1967  gdpPercap_1972  gdpPercap_1977  \
country                                                                  
Albania                    2760.196931     3313.422188     3533.003910   
Austria                   12834.602400    16661.625600    19749.422300   
Belgium                   13149.041190    16672.143560    19117.974480   
Bosnia and Herzegovina     2172.352423     2860.169750     3528.481305   
Bulgaria                   5577.002800     6597.494398     7612.240438   

                        gdpPercap_1982  gdpPercap_1987  gdpPercap_1992  \
country                                                                  
Albania                    3630.880722     3738.932735     2497.437901   
Austria                   21597.083620    23687.826070    27042.018680   
Belgium                   20979.845890    22525.563080    25575.570690   
Bosnia and Herzegovina     4126.613157     4314.114757     2546.781445   
Bulgaria                   8224.191647     8239.854824     6302.623438   

                        gdpPercap_1997  gdpPercap_2002  gdpPercap_2007  
country                                                                 
Albania                    3193.054604     4604.211737     5937.029526  
Austria                   29095.920660    32417.607690    36126.492700  
Belgium                   27561.196630    30485.883750    33692.605080  
Bosnia and Herzegovina     4766.355904     6018.975239     7446.298803  
Bulgaria                   5970.388760     7696.777725    10680.792820

- When data is in *long format*, multiple columns
  are melted into a single columns and entries are repeated.
- To change data to *long format*, `melt()` the columns

In [16]:
# Country is not currently a variable,
# but an index
long = data.reset_index()
long = pd.melt(long, id_vars='country', var_name='GDP_Year').sort_values(by='country')
long

country        GDP_Year         value
0           Albania  gdpPercap_1952   1601.056136
180         Albania  gdpPercap_1982   3630.880722
240         Albania  gdpPercap_1992   2497.437901
150         Albania  gdpPercap_1977   3533.003910
120         Albania  gdpPercap_1972   3313.422188
270         Albania  gdpPercap_1997   3193.054604
90          Albania  gdpPercap_1967   2760.196931
300         Albania  gdpPercap_2002   4604.211737
60          Albania  gdpPercap_1962   2312.888958
330         Albania  gdpPercap_2007   5937.029526
30          Albania  gdpPercap_1957   1942.284244
210         Albania  gdpPercap_1987   3738.932735
211         Austria  gdpPercap_1987  23687.826070
121         Austria  gdpPercap_1972  16661.625600
271         Austria  gdpPercap_1997  29095.920660
61          Austria  gdpPercap_1962  10750.721110
31          Austria  gdpPercap_1957   8842.598030
181         Austria  gdpPercap_1982  21597.083620
331         Austria  gdpPercap_2007  36126.492700
151         Austria  gdpPercap_1977  19749.422300
301         Austria  gdpPercap_2002  32417.607690
1           Austria  gdpPercap_1952   6137.076492
91          Austria  gdpPercap_1967  12834.602400
241         Austria  gdpPercap_1992  27042.018680
122         Belgium  gdpPercap_1972  16672.143560
212         Belgium  gdpPercap_1987  22525.563080
272         Belgium  gdpPercap_1997  27561.196630
92          Belgium  gdpPercap_1967  13149.041190
62          Belgium  gdpPercap_1962  10991.206760
32          Belgium  gdpPercap_1957   9714.960623
..              ...             ...           ...
87      Switzerland  gdpPercap_1962  20431.092700
267     Switzerland  gdpPercap_1992  31871.530300
207     Switzerland  gdpPercap_1982  28397.715120
117     Switzerland  gdpPercap_1967  22966.144320
57      Switzerland  gdpPercap_1957  17909.489730
27      Switzerland  gdpPercap_1952  14734.232750
118          Turkey  gdpPercap_1967   2826.356387
358          Turkey  gdpPercap_2007   8458.276384
178          Turkey  gdpPercap_1977   4269.122326
328          Turkey  gdpPercap_2002   6508.085718
238          Turkey  gdpPercap_1987   5089.043686
148          Turkey  gdpPercap_1972   3450.696380
298          Turkey  gdpPercap_1997   6601.429915
88           Turkey  gdpPercap_1962   2322.869908
208          Turkey  gdpPercap_1982   4241.356344
268          Turkey  gdpPercap_1992   5678.348271
28           Turkey  gdpPercap_1952   1969.100980
58           Turkey  gdpPercap_1957   2218.754257
179  United Kingdom  gdpPercap_1977  17428.748460
299  United Kingdom  gdpPercap_1997  26074.531360
269  United Kingdom  gdpPercap_1992  22705.092540
239  United Kingdom  gdpPercap_1987  21664.787670
209  United Kingdom  gdpPercap_1982  18232.424520
149  United Kingdom  gdpPercap_1972  15895.116410
119  United Kingdom  gdpPercap_1967  14142.850890
89   United Kingdom  gdpPercap_1962  12477.177070
59   United Kingdom  gdpPercap_1957  11283.177950
29   United Kingdom  gdpPercap_1952   9979.508487
329  United Kingdom  gdpPercap_2002  29478.999190
359  United Kingdom  gdpPercap_2007  33203.261280

[360 rows x 3 columns]

- `pivot()` the data to go back to *wide format*

In [17]:
long.pivot(index='country', columns='GDP_Year')

value                                \
GDP_Year               gdpPercap_1952 gdpPercap_1957 gdpPercap_1962   
country                                                               
Albania                   1601.056136    1942.284244    2312.888958   
Austria                   6137.076492    8842.598030   10750.721110   
Belgium                   8343.105127    9714.960623   10991.206760   
Bosnia and Herzegovina     973.533195    1353.989176    1709.683679   
Bulgaria                  2444.286648    3008.670727    4254.337839   
Croatia                   3119.236520    4338.231617    5477.890018   
Czech Republic            6876.140250    8256.343918   10136.867130   
Denmark                   9692.385245   11099.659350   13583.313510   
Finland                   6424.519071    7545.415386    9371.842561   
France                    7029.809327    8662.834898   10560.485530   
Germany                   7144.114393   10187.826650   12902.462910   
Greece                    3530.690067    4916.299889    6017.190733   
Hungary                   5263.673816    6040.180011    7550.359877   
Iceland                   7267.688428    9244.001412   10350.159060   
Ireland                   5210.280328    5599.077872    6631.597314   
Italy                     4931.404155    6248.656232    8243.582340   
Montenegro                2647.585601    3682.259903    4649.593785   
Netherlands               8941.571858   11276.193440   12790.849560   
Norway                   10095.421720   11653.973040   13450.401510   
Poland                    4029.329699    4734.253019    5338.752143   
Portugal                  3068.319867    3774.571743    4727.954889   
Romania                   3144.613186    3943.370225    4734.997586   
Serbia                    3581.459448    4981.090891    6289.629157   
Slovak Republic           5074.659104    6093.262980    7481.107598   
Slovenia                  4215.041741    5862.276629    7402.303395   
Spain                     3834.034742    4564.802410    5693.843879   
Sweden                    8527.844662    9911.878226   12329.441920   
Switzerland              14734.232750   17909.489730   20431.092700   
Turkey                    1969.100980    2218.754257    2322.869908   
United Kingdom            9979.508487   11283.177950   12477.177070   

                                                                     \
GDP_Year               gdpPercap_1967 gdpPercap_1972 gdpPercap_1977   
country                                                               
Albania                   2760.196931    3313.422188    3533.003910   
Austria                  12834.602400   16661.625600   19749.422300   
Belgium                  13149.041190   16672.143560   19117.974480   
Bosnia and Herzegovina    2172.352423    2860.169750    3528.481305   
Bulgaria                  5577.002800    6597.494398    7612.240438   
Croatia                   6960.297861    9164.090127   11305.385170   
Czech Republic           11399.444890   13108.453600   14800.160620   
Denmark                  15937.211230   18866.207210   20422.901500   
Finland                  10921.636260   14358.875900   15605.422830   
France                   12999.917660   16107.191710   18292.635140   
Germany                  14745.625610   18016.180270   20512.921230   
Greece                    8513.097016   12724.829570   14195.524280   
Hungary                   9326.644670   10168.656110   11674.837370   
Iceland                  13319.895680   15798.063620   19654.962470   
Ireland                   7655.568963    9530.772896   11150.981130   
Italy                    10022.401310   12269.273780   14255.984750   
Montenegro                5907.850937    7778.414017    9595.929905   
Netherlands              15363.251360   18794.745670   21209.059200   
Norway                   16361.876470   18965.055510   23311.349390   
Poland                    6557.152776    8006.506993    9508.141454   
Portugal                  6361.517993    9022.247417   

## Objectives
- Select individual values from a Pandas dataframe.
- Select entire rows or entire columns from a dataframe.
- Select a subset of both rows and columns from a dataframe in a single operation.
- Filter data based on values in a dataframe.
- Apply a split-apply-combine workflow to a dataframe.
- Convert a dataframe between wide and long data formats.